# <i>CatBoost learning to rank on Microsoft dataset</i>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/catboost/tutorials/blob/master/ranking/ranking_tutorial.ipynb)

In [3]:
%pip install catboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.7 MB/s eta 0:00:00


In [4]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import numpy as np
import os
import pandas as pd

### Ranking problem

Let's introduce a notation:
* Let $Q = \{q_1, \dots, q_n\}$ be the set of groups
* $D_q = \{d_{q1}, \dots, d_{qm}\}$ -- set of objects retrieved for a group $q$
* $L_q = \{l_{q1}, \dots, l_{qm}\}$ -- relevance labels for the objects from the set $D_q$

Every object $d_{qi}$ is represented in the vector space of features, describing the associatinos between the group and the object.

So every group is associated with set of objects. For example, group is a query and object is a document if we are ranking documents for a search query.

The goal is to learn the ranking function $f = f(d_{qi})$, such that the ranking of objects $d_{qi}$ for all groups from $Q$ based on their scores $x_{qi} = f(d_{qi})$, is as close as possible to the ideal ranking from the editorial judgements $l_{qi}$.

### Ranking quality metrics:
* __Precision__
    $$ \mbox{P}=\frac{|\{\mbox{relevant docs}\}\cap\{\mbox{retrieved docs}\}|}{|\{\mbox{retrieved docs}\}|} $$
* __Recall__
    $$ \mbox{R}=\frac{|\{\mbox{relevant docs}\}\cap\{\mbox{retrieved docs}\}|}{|\{\mbox{relevant docs}\}|} $$
    
    Notation $@k$ means that metric is calculated on the first $k$ documents from ranking list.

    For example, if 1,2,5,7,9 is the ranks of relevant documents (enumerations starts from number 1) from ten retrivied then $P@5$ will be $\frac{3}{5}$.

* __Mean average precision (MAP)__
    $$\frac{1}{|Q|}\sum_{q \in Q} \frac{1}{|\mbox{relevant docs in } D_q|} \sum_{k} P@k(q) \times rel(q, k) $$
    
    Where $rel(q, k)$ is a relevance label of the document at k-th position in our ranking of $D_q$. This metric calculates average precision for a query weighted with document relevances and then calculate mean between all queries.
    
* __Discounted cumulative gain (DCG)__
    $$\sum_{k=1}^{mq} \frac{2 ^ {l_{qk}}}{\log_2(k+1)}$$
    
    This metric takes into account user behavior: user attention is high on the top and then nonlinear decrease to the end.
    
* __NDCG__ - normalized DCG = DCG $~ / ~$ IDCG, where IDCG is a maximum possible value of DCG with given set of relevance labels.

* __AverageGain__ - represents the average value of the label values for objects with the defined top  label values.

* __[PFound](https://tech.yandex.com/catboost/doc/dg/references/pfound-docpage/#pfound)__
    
More on wiki: https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

Parameter $@k$ for every metric can be specified through metric parameter "top", for example "NDCG:top=10", would mean NDCG@10.

### Download part of [MSRank](https://www.microsoft.com/en-us/research/project/mslr/) dataset from CatBoost datasets storage

In [5]:
from catboost.datasets import msrank_10k
train_df, test_df = msrank_10k()

X_train = train_df.drop([0, 1], axis=1).values
y_train = train_df[0].values
queries_train = train_df[1].values

X_test = test_df.drop([0, 1], axis=1).values
y_test = test_df[0].values
queries_test = test_df[1].values

In [8]:
queries_train

array([   1,    1,    1, ..., 1291, 1291, 1291])

In [7]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,2.0,1,3,3,0,0,3,1.0,1.0,0.000000,...,62,11089534,2,116,64034,13,3,0,0,0.0
1,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,54,11089534,2,124,64034,1,2,0,0,0.0
2,0.0,1,3,0,2,0,3,1.0,0.0,0.666667,...,45,3,1,124,3344,14,67,0,0,0.0
3,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,56,11089534,13,123,63933,1,3,0,0,0.0
4,1.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,64,5,7,256,49697,1,13,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,1291,2,0,1,0,2,1.0,0.0,0.500000,...,22,131,2,24231,4897,95,228,0,0,0.0
9996,0.0,1291,2,0,1,1,2,1.0,0.0,0.500000,...,39,0,1,52316,55891,79,124,0,0,0.0
9997,2.0,1291,2,0,2,0,2,1.0,0.0,1.000000,...,14,11040,6,44336,26674,81,153,0,0,0.0
9998,2.0,1291,2,0,1,0,2,1.0,0.0,0.500000,...,31,105,0,10394,4079,34,155,0,0,0.0


In [6]:
X_train

array([[3., 3., 0., ..., 0., 0., 0.],
       [3., 0., 3., ..., 0., 0., 0.],
       [3., 0., 2., ..., 0., 0., 0.],
       ...,
       [2., 0., 2., ..., 0., 0., 0.],
       [2., 0., 1., ..., 0., 0., 0.],
       [2., 1., 1., ..., 0., 0., 0.]])

### Dataset analysis

__Number of documents__

In [ ]:
num_documents = X_train.shape[0]
print(num_documents)

__Number of features__

In [ ]:
X_train.shape[1]

__Relevance labels statistics__

0 - irrelevant, 1 - highly relevant. Table represents number of documents for each value.

In [ ]:
from collections import Counter
Counter(y_train).items()

For calculation such metrics as NDCG and PFound relevances should be in segment \[0,1\].

In [ ]:
max_relevance = np.max(y_train)
y_train /= max_relevance
y_test /= max_relevance

__Number of queries__

In [ ]:
num_queries = np.unique(queries_train).shape[0]
num_queries

### Creation of CatBoost pools

In [ ]:
train = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

test = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)

### You can also create pools from files

In [ ]:
data_dir = './msrank'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

train_file = os.path.join(data_dir, 'train.csv')
test_file = os.path.join(data_dir, 'test.csv')

train_df.to_csv(train_file, index=False, header=False)
test_df.to_csv(test_file, index=False, header=False)

In [ ]:
description_file = os.path.join(data_dir, 'dataset.cd')
with open(description_file, 'w') as f:
    f.write('0\tLabel\n')
    f.write('1\tQueryId\n')

In [ ]:
Pool(data=train_file, column_description=description_file, delimiter=',')

### <span style="color:#ce2029">Attention:</span> all objects in dataset must be grouped by group_id

For example, if the dataset consits of five documents
\[d1, d2, d3, d4, d5\] with corresponding queries \[q1, q2, q2, q1, q2\] then the dataset should be look like:

$$\begin{pmatrix}
    d_1, q_1, f_1\\
    d_4, q_1, f_4\\
    d_2, q_2, f_2\\
    d_3, q_2, f_3\\
    d_5, q_2, f_5\\
\end{pmatrix} \hspace{6px} \texttt{or} \hspace{6px}
\begin{pmatrix}
    d_2, q_2, f_2\\
    d_3, q_2, f_3\\
    d_5, q_2, f_5\\
    d_1, q_1, f_1\\
    d_4, q_1, f_4\\
\end{pmatrix}$$

where $f_i$ is feature vector of i-th document.

### Reducing problem to machine learning task

The first and simplest idea is to try predicting document relevance $l_q$ minimizing RMSE.

$$\frac{1}{N}\sqrt{ \sum_q \sum_{d_{qk}} \left(f(d_{qk}) - l_{qk} \right)^2 }$$

In [ ]:
default_parameters = {
    'iterations': 2000,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10'],
    'verbose': False,
    'random_seed': 0,
}

parameters = {}

In [ ]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function

    if additional_params is not None:
        parameters.update(additional_params)

    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)

    return model

Lets train the simplest model and also demonstrate precision/recall metrics from introduction.

In [ ]:
model = fit_model('RMSE', {'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10']})

### Group weights parameter
Suppose we know that some queries are more important than others for us.<br/>
The word "importance" used here in terms of accuracy or quality of CatBoostRanker prediction for given queries.<br/>
You can pass this additional information for learner using a ``group_weights`` parameter.<br/>
Under the hood, CatBoostRanker uses this weights in loss function simply multiplying it on a group summand.<br/>
So the bigger weight $\rightarrow$ the more attention for query.<br/>
Let's show an example of training procedure with random query weights.

In [ ]:
def create_weights(queries):
    query_set = np.unique(queries)
    query_weights = np.random.uniform(size=query_set.shape[0])
    weights = np.zeros(shape=queries.shape)

    for i, query_id in enumerate(query_set):
        weights[queries == query_id] = query_weights[i]

    return weights


train_with_weights = Pool(
    data=X_train,
    label=y_train,
    group_weight=create_weights(queries_train),
    group_id=queries_train
)

test_with_weights = Pool(
    data=X_test,
    label=y_test,
    group_weight=create_weights(queries_test),
    group_id=queries_test
)

fit_model(
    'RMSE',
    additional_params={'train_dir': 'RMSE_weigths'},
    train_pool=train_with_weights,
    test_pool=test_with_weights
)

### A special case: top-1 prediction

Someday you may face with a problem $-$ you will need to predict the top one most relevant object for a given query.<br/>
For this purpose CatBoostRanker has a mode called __QuerySoftMax__.

Suppose our dataset contain a binary target: 1 $-$ mean best document for a query, 0 $-$ others.<br/>
We will maximize the probability of being the best document for given query.<br/>
MSRANK dataset doesn't contain binary labels, but for example of method __QuerySoftMax__ we convert it to that format,<br/> choosing a best document for every query.

In [ ]:
def get_best_documents(labels, queries):
    query_set = np.unique(queries)
    num_queries = query_set.shape[0]
    by_query_arg_max = {query: -1 for query in query_set}

    for i, query in enumerate(queries):
        best_idx = by_query_arg_max[query]
        if best_idx == -1 or labels[best_idx] < labels[i]:
            by_query_arg_max[query] = i

    binary_best_docs = np.zeros(shape=labels.shape)
    for arg_max in by_query_arg_max.values():
        binary_best_docs[arg_max] = 1.

    return binary_best_docs

In [ ]:
best_docs_train = get_best_documents(y_train, queries_train)
best_docs_test = get_best_documents(y_test, queries_test)

train_with_weights = Pool(
    data=X_train,
    label=best_docs_train,
    group_id=queries_train,
    group_weight=create_weights(queries_train)
)

test_with_weights = Pool(
    data=X_test,
    label=best_docs_test,
    group_id=queries_test,
    group_weight=create_weights(queries_test)
)

fit_model(
    'QuerySoftMax',
    additional_params={'custom_metric': 'AverageGain:top=1'},
    train_pool=train_with_weights,
    test_pool=test_with_weights
)

### Reducing ploblem, step 2

Now lets look at example of documents relevance:

$$
    \begin{align}
    labels(q_1) &= \begin{bmatrix}
           4 \\
           3 \\
           3 \\
           1
         \end{bmatrix},
    labels(q_2) &= \begin{bmatrix}
           2 \\
           1 \\
           1 \\
           0
         \end{bmatrix}
   \end{align}
$$

This means that with RMSE loss function we pay more attention to q1 than q2.

To avoid this problem we introduce into RMSE a coefficient $c_q$ which depends only on query (and if fact equals to the mean of the difference between prediction and label).

$$\frac{1}{N}\sqrt{ \sum_q \sum_{d_{qk}} \left(f(d_{qk}) - l_{qk} - \color{red}{c_{q}} \right)^2 }$$

In [ ]:
fit_model('QueryRMSE')

### Reducing problem, step 3

Since the goal of ranking is to predict a list of documents (which can be generated from given document relevances) RMSE loss function doesn't take into account relations between documents: the first is better than second, second is better than third and fifth etc.

We can easily bring this information into the loss function, reducing problem not to regression but classification for two documents $(d_i, d_j)$ -- does $i$th better than $j$th or not.

So we minimize the negative loglikelihood:

$$ - \sum_{i,j \in Pairs} \log \left( \frac{1}{1 + \exp{-(f(d_i) - f(d_j))}} \right) $$

Methods based on pair comparisons called __pairwise__ in CatBoostRanker this objective called __PairLogit__.

There's no need to change the dataset CatBoost generate the pairs for us. The number of generating pairs managed via parameter max_size.

In [ ]:
fit_model('PairLogit')

Also we can to specify the pairs directly. There are two ways to do that:

1. Two-dimensional matrix with shape=(num_pairs, 2) $\rightarrow$ (winner_id, loser_id): list, numpy.ndarray, pandas.DataFrame.
2. Path two the input file that contains pair descriptions:
    * Row format: $\texttt{[winner index, loser index, pair weight]}$

In [ ]:
def read_groups(file_name):
    groups = {}
    group_ids = []

    with open(file_name) as f:
        for doc_id, line in enumerate(f):
            line = line.split(',')[:2]

            label, query_id = float(line[0]), int(line[1])
            if query_id not in groups:
                groups[query_id] = []
            groups[query_id].append((doc_id, label))

            group_ids.append(query_id)

    return groups, group_ids

train_groups, train_group_ids = read_groups(train_file)
assert num_queries == len(train_groups)

In [ ]:
pairs = []

for group in train_groups.values():
    for i in range(len(group)):
        for j in range(i, len(group)):
            if i == j:
                continue
            doc_i, relevance_i = group[i]
            doc_j, relevance_j = group[j]
            if relevance_i < relevance_j:
                pairs.append((doc_j, doc_i))
            else:
                pairs.append((doc_i, doc_j))

pairs_file = os.path.join(data_dir, 'pairs.csv')

with open(pairs_file, 'w') as f:
    for pair in pairs:
        f.write(str(pair[0]) + '\t' + str(pair[1]) + '\t1\n')

In [ ]:
pool1 = Pool(data=X_train, label=y_train, group_id=train_group_ids, pairs=pairs)
pool2 = Pool(data=train_file, column_description=description_file, pairs=pairs_file, delimiter=',')

### Reducing problem, step 3.1

Thus we know that $f(d_{qk})$ is a ensemble of trees, we can accurately solve the minimization task from step 3.

This method called __PairLogitPairwise__.

In [ ]:
fit_model('PairLogitPairwise')

### Reducing problem, step 4

Previous loss function directly minimize the number of pairs $(d_i, d_j)$ where $l_i > l_j$ but $f(d_i) < f(d_j)$, simply said the number of incorrectly placed documents.

Since the user attention is high on the first documents and low on last the incorrect switch of the first two documents and last two has different cost.

In steps 3 and 3.1 user can set the weight for pair.

Method __YetiRank__ take this effect into account and generates weights for pairs according to their positions ([paper](https://cache-mskstoredata08.cdn.yandex.net/download.yandex.ru/company/to_rank_challenge_with_yetirank.pdf)).

$$ - \sum_{i,j \in Pairs} \color{red}{w_{ij}} \log \left( \frac{1}{1 + \exp{-(f(d_i) - f(d_j))}} \right) $$

In [ ]:
fit_model('YetiRank')

### Step 4.1

As in step 3.1 __YetiRankPairwise__ is slower than __YetiRank__, but gives more accurate results.

In [ ]:
fit_model('YetiRankPairwise')

In [ ]:
widget = MetricVisualizer(['RMSE', 'QueryRMSE', 'PairLogit', 'PairLogitPairwise', 'YetiRank', 'YetiRankPairwise'])
widget.start()

### Simple classification

Very fast $\rightarrow$ very slow; Simple method $\rightarrow$ complex method; Low quality $\rightarrow$ high quality.

1. RMSE
2. QueryRMSE
3. PairLogit
4. PairLogitPairwise
5. YetiRank
6. YetiRankPairwise

Besides our classification, the quality of the concrete method may depend on your dataset.

Look on NDCG metric of method YetiRank $-$ it's underfitted.

In [ ]:
fit_model('YetiRank', {'train_dir': 'YetiRank-lr-0.3', 'learning_rate': 0.3})

In [ ]:
widget = MetricVisualizer(['YetiRank', 'YetiRank-lr-0.3'])
widget.start()

### Additional parameters

__Metric period__

Period in iterations of calculation metrics. This parameter can speed up training process.

In [ ]:
fit_model('YetiRank', {'metric_period': 50})

__Task type__

You can significantly speed up training procedure by switching to GPU.

In [ ]:
fit_model('YetiRank', {'task_type': 'GPU'})